# Concours sur les prix de vente immobiliers

Avec la médianne

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
import pandas as pd

# Fonction pour effectuer la validation croisée avec GridSearchCV
def cross_validate_model(pipeline, X_train, y_train, param_grid):
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error')
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_, grid_search.best_params_, grid_search.best_score_

# Charger les données
train_path = 'train.csv'  # Remplacez par le chemin correct
X_full = pd.read_csv(train_path, index_col='Id')

test_path = 'test.csv'  # Remplacez par le chemin correct
X_test_full = pd.read_csv(test_path, index_col='Id')

# Prétraitement initial
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full['SalePrice']
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Séparer les données d'entraînement et de validation
X_train_full, X_valid_full, y_train, y_valid = train_test_split(
    X_full, y, train_size=0.8, test_size=0.2, random_state=0
)

# Identifier les colonnes numériques et catégoriques
categorical_cols = [
    cname for cname in X_train_full.columns
    if X_train_full[cname].dtype == "object" and X_train_full[cname].nunique() < 10
]
numerical_cols = [
    cname for cname in X_train_full.columns
    if X_train_full[cname].dtype in ['int64', 'float64']
]

# Garder les colonnes sélectionnées
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# Définir les transformations pour les colonnes numériques et catégoriques
numerical_transformer = SimpleImputer(strategy='median')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combiner les transformations dans un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Définir le modèle XGBoost
model_xgb = XGBRegressor(random_state=0)

# Créer le pipeline avec XGBoost
my_pipeline_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model_xgb)
])

# Définir la grille des hyperparamètres pour XGBoost
param_grid_xgb = {
    'model__n_estimators': [50, 100, 150],
    'model__max_depth': [3, 5, 7],
    'model__learning_rate': [0.01, 0.1, 0.2],
    'model__subsample': [0.8, 1.0]
}

# Appliquer la validation croisée pour trouver le meilleur modèle XGBoost
best_model_xgb, best_params_xgb, best_score_xgb = cross_validate_model(my_pipeline_xgb, X_train, y_train, param_grid_xgb)

# Afficher les résultats
print("Meilleur modèle XGBoost :", best_model_xgb)
print("Meilleurs paramètres XGBoost :", best_params_xgb)
print("Meilleur score MAE XGBoost :", -best_score_xgb)


Meilleur modèle XGBoost : Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='median'),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBath...
                       

In [17]:
# Faire des prédictions sur les données de test avec le meilleur modèle
preds_test_xgb = best_model_xgb.predict(X_test)

# Préparer le fichier de soumission
output = pd.DataFrame({'Id': X_test.index, 'SalePrice': preds_test_xgb})
output.to_csv('submission_xgb.csv', index=False)

# Afficher un message de confirmation
print("Fichier de soumission créé : submission_xgb.csv")


Fichier de soumission créé : submission_xgb.csv
